In [10]:
import numpy as np
import numexpr as ne
import glob
from helpers.io_utils import hlist2pandas
from astropy.io import ascii
from astropy.table import Column, Table
import pandas as pd
from scipy.stats import binned_statistic


In [2]:
particle_mass = 1.3e8 ##M_sun/h## 
h = 0.7
crit_density = 94*2.7754e11*h**2 ##M_sun/Mpc**3##

halo_names = []
host_ids = []
with open('halos_info_2.txt') as f:
    for l in f:
        this_halo, host_id, block, _ = l.split()
        halo_names.append(this_halo)
        host_ids.append(host_id)


In [3]:
def calc_density(r,mass):
    V = (4.0/3.0)*np.pi*r**3
    density = mass/V

    return density

def calc_density_iter(r,particle_mass,crit_density):
    
    r = np.sort(r)*1e-3

    for i in range(len(r)):
        
        mass = (i+1)*particle_mass*h
        density = calc_density(r[i],mass)
        
        if density > crit_density:
            continue
        else:
            return r[i]

In [42]:

counts, bin_edges = np.histogram(r,bins=50)
density = 10000
i = 0
while density > 1:
    r_set = r[np.logical_and(r>bin_edges[i],r<bin_edges[i+1])]
    print(len(r_set))
    mass = len(r_set)*10000
    density = calc_density(bin_edges[i+1],mass)
    print(bin_edges[i],density)
    i+=1

3
0.0 113.60133346359984
4
3.98 18.933555577266638
4
7.96 5.60994239326419
4
11.94 2.3666944471583298
4
15.92 1.211747556945065
4
19.9 0.7012427991580238


In [22]:
j = 0
for f in halo_names:
    print(f)

    # load host    
    path = '/home/lom31/rhap_particles/rhapsody/{}/rockstar/'.format(f)
    hostvalues = hlist2pandas(path + '/out_199.list')
    #hostvalues.rename(columns={c: c.lower() for c in hostvalues.columns})
    hostvalues = Table.from_pandas(hostvalues)

    fname = '/home/lom31/rhap_particles/particle_tables/{}_all.particle_table'.format(f)
    particlevalues = ascii.read(fname, format = 'commented_header')
    ##Make sure chosen particles are within the virial radius##
    r = Column(np.sqrt(particlevalues['x']**2+particlevalues['y']**2+particlevalues['z']**2),name='r')

    rvir = calc_density_iter(r,particle_mass,crit_density)

    print(rvir)
    

Halo156


FileNotFoundError: [Errno 2] No such file or directory: '/home/lom31/rhap_particles/rhapsody/Halo156/rockstar//out_199.list'